In [4]:
from functools import partial
from hyperopt import Trials, fmin, tpe, hp, STATUS_OK
import tensorflow as tf
from tensorflow.keras import backend as K
import datetime

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D


import mlflow
import mlflow.tensorflow
mlflow.tensorflow.autolog(every_n_iter=1)

In [5]:
tf.__version__

'1.15.4'

In [6]:
# You would have to expose this containers port also
#remote_server_uri = "mlruns" # set to your server URI
remote_server_uri = "http://10.0.0.115:5000"
mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_experiment("my-hypteropt-experiment3")

In [ ]:
def f1_score(y_true, y_pred):
    # returns batch-wise averge f1, not weighted
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


def train(constants, hyperparams):

    #mlflow.start_run()
    with mlflow.start_run():
        params = {**constants, **hyperparams}

        batch_size = int(params['batch_size'])
        num_classes = int(params['num_classes'])
        epochs = int(params['epochs'])
        lr = float(params['lr'])
        optimizer_str = params['optimizer_str']

        # input image dimensions
        img_rows, img_cols = 28, 28
        input_shape = (img_rows, img_cols, 1)

        # the data, split between train and test sets
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255

        # convert class vectors to binary class matrices
        y_train = tf.keras.utils.to_categorical(y_train, num_classes)
        y_test = tf.keras.utils.to_categorical(y_test, num_classes)

        callbacks = [tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=5  )]

        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3),
                         activation='relu',
                         input_shape=input_shape))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(loss=tf.keras.losses.categorical_crossentropy,
                      optimizer=optimizer_str,
                      # user defined metrics that get reported
                      metrics=['accuracy', f1_score])

        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=2,
                            callbacks=callbacks,
                            validation_data=(x_test, y_test))

    #mlflow.end_run()
    return {'loss': min(history.history['val_f1_score']), 'status': STATUS_OK}


# Keep parallelism at or below number of hyperparams
search_space = {
    'lr': hp.uniform('lr', 1e-6, 1e-4),
    'optimizer_str': hp.choice('optimizer_str', ['RMSprop', 'Adam', 'Adadelta'])
}

constants = {
    'batch_size': 128,
    'epochs': 10,
    'num_classes': 10
}

#hyperopt

train_func = partial(train, constants)

# using default experiment
best = fmin(train_func, space=search_space,
            algo=tpe.suggest, max_evals=10, trials=Trials())

Train on 60000 samples, validate on 10000 samples     
Epoch 1/10                                            
60000/60000 - 32s - loss: 0.2396 - acc: 0.9277 - f1_score: 0.9205 - val_loss: 0.0486 - val_acc: 0.9847 - val_f1_score: 0.9848

Epoch 2/10                                            
60000/60000 - 31s - loss: 0.0852 - acc: 0.9742 - f1_score: 0.9745 - val_loss: 0.0434 - val_acc: 0.9847 - val_f1_score: 0.9853

Epoch 3/10                                            
60000/60000 - 31s - loss: 0.0667 - acc: 0.9803 - f1_score: 0.9806 - val_loss: 0.0347 - val_acc: 0.9893 - val_f1_score: 0.9894

Epoch 4/10                                            
60000/60000 - 31s - loss: 0.0523 - acc: 0.9837 - f1_score: 0.9839 - val_loss: 0.0289 - val_acc: 0.9905 - val_f1_score: 0.9904

Epoch 5/10                                            
60000/60000 - 31s - loss: 0.0450 - acc: 0.9862 - f1_score: 0.9863 - val_loss: 0.0300 - val_acc: 0.9901 - val_f1_score: 0.9902

Epoch 6/10                         

In [8]:

best

{'lr': 3.989865204889641e-05, 'optimizer_str': 2}

In [11]:
hp.choice('optimizer_str', ['RMSprop', 'Adam', 'Adadelta'])['optimizer_str']